In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
from common import *
from PatientVec.Experiments.hyperparam_exps import *

In [5]:
import argparse
parser = argparse.ArgumentParser(description='Run Diagnosis experiments')
parser.add_argument("--data_dir", type=str, required=True)
parser.add_argument('--display', dest='display', action='store_true')
parser.add_argument("--output_dir", type=str)
parser.add_argument("--mock", dest='mock', action='store_true')

args = parser.parse_args(['--data_dir=.', '--output_dir=outputs/', '--display'])

In [6]:
from dataloaders import mortality_dataset
data = mortality_dataset(args, _type='30day')

INFO - 2019-03-19 13:56:49,101 - Reading Structured data ...
INFO - 2019-03-19 13:56:49,191 - Reading Notes ...
INFO - 2019-03-19 13:57:06,035 - Stratifying ...


In [7]:
train_data, dev_data = get_basic_data(data, structured=True, truncate=90)

Pos Percentage [0.15129873]


INFO - 2019-03-19 13:57:13,242 - Maximum Sentence Length 721825.000000 , 90 percentile length 18596.000000 ... 
INFO - 2019-03-19 13:57:28,195 - Truncated all ...


Pos Percentage [0.14555348]


INFO - 2019-03-19 13:57:30,176 - Maximum Sentence Length 420764.000000 , 90 percentile length 17662.000000 ... 
INFO - 2019-03-19 13:57:32,888 - Truncated all ...


In [10]:
config = {'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name, 
          'type' : data.metrics_type, 'norm' : 'l2', 'constant_mul' : 1.0}
lr = LR(config)
lr.train(train_data)
metrics = lr.evaluate(dev_data, save_results=True)


Normalising using l2
Using Norm from linalg
Multiplying by constant ,  1.0
Fit BOW Classifier ...
Fit BOW Structured Classifier ...



Normalising using l2
Using Norm from linalg
Multiplying by constant ,  1.0
BOW
{'accuracy': 0.8750212548886244, 'roc_auc': 0.9290389175617242, 'pr_auc': 0.7844204587167638}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.924,0.656,0.875,0.790,0.885
precision,0.967,0.547,0.875,0.757,0.905
recall,0.884,0.820,0.875,0.852,0.875
support,5025.000,856.000,5881.000,5881.000,5881.000


BOW+Structured
{'accuracy': 0.8787621152865159, 'roc_auc': 0.9389143069698238, 'pr_auc': 0.8008571942720953}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.926,0.669,0.879,0.797,0.888
precision,0.970,0.555,0.879,0.763,0.910
recall,0.885,0.840,0.879,0.863,0.879
support,5025.000,856.000,5881.000,5881.000,5881.000


Saving Models
==============

In [ ]:
from common import generate_latex_tables
keys_to_use = ['roc_auc', 'pr_auc']
generate_latex_tables(data, keys_to_use)

In [ ]:
dirname = 'outputs/Readmission/Basic/'
exps = os.listdir(dirname)
for e in sorted(exps) :
    if 'Structured' in e :
        print(e)
        print_results_from_model(get_latest_model(os.path.join(dirname, e)))

In [ ]:
dirname = 'outputs/Diagnosis/Basic/'
exps = os.listdir(dirname)
for e in sorted(exps) :
    if 'Structured' in e :
        print(e)
        print_results_from_model(get_latest_model(os.path.join(dirname, e)))

In [5]:
keys = ['roc_auc', 'pr_auc']
yr_df = defaultdict(list)
for yr in [1] :
    for model in ['baselines', 'Basic', 'Attention', 'Structured Attention'] :
        name = os.path.join('outputs/Mortality_30day/', model)
        dirs = os.listdir(name)
        dfs = {}
        for d in sorted(dirs) :
            dirname = os.path.join(name, d)
            model_name = get_latest_model(dirname)
            if model_name is not None :
                df = json.load(open(os.path.join(model_name, 'evaluate.json')))
                dfs[d] = {k:v for k, v in df.items() if k in keys}
        
        dfs = pd.DataFrame(dfs).transpose()
        yr_df[yr] += [dfs]
        
    yr_df[yr] = pd.concat(yr_df[yr], axis=0)
yr_df = pd.concat(yr_df.values(), axis=1, keys=yr_df.keys()).round(3)
yr_df.columns = yr_df.columns.swaplevel(0, 1)
yr_df.sort_index(axis=1, level=0, inplace=True)
display(HTML(yr_df.to_html()))

,pr_auc,roc_auc
,1,1
LR+BOW+norm=l1+clip=False,0.592,0.862
LR+BOW+norm=l2+clip=False,0.808,0.931
LR+BOW+norm=l3+clip=False,0.794,0.923
Average(hs=256),0.743,0.907
Average(hs=256)+Structured,0.622,0.886
"CNN(hs=64)(kernels=3,5,7,9)",0.789,0.928
"CNN(hs=64)(kernels=3,5,7,9)+Structured",0.775,0.928
LSTM(hs=128),0.802,0.923
LSTM(hs=128)+Structured,0.789,0.926
